##General Setup

Imports and persistant storage in google drive.

In [3]:
from google.colab import drive, userdata
import pandas as pd
import json
import os
from tqdm import tqdm
import json
from pathlib import Path


In [4]:

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Set relevant directories

In [5]:
EXTENSION_DIR = '/content/drive/MyDrive/model_2.0/model'
VIDEO_OUT = EXTENSION_DIR + '/uncut_videos'
VIDEO_OUT_new = '/content/drive/MyDrive/model_2.0/model/uncut_videos_6tol/v1/clips'
VIDEO_OUT_new_cut = '/content/drive/MyDrive/model_2.0/model/cut_videos_6tol'
UNCUT_VIDEO_DIR = VIDEO_OUT + '/v1/clips'
CUT_VIDEO_OUT = EXTENSION_DIR + '/cut_videos'

## Implement and train model

Set annotation path and replace NaN values.

In [6]:
annotations_path = f'{EXTENSION_DIR}/GroundTruth.xlsx'
annotations_df = pd.read_excel(annotations_path)

annotations_df.head()

,idx,annotation_uid,clip_uid,sentence,predicted_times,exact_times,video_uid,ground truth
0,0,93891b44-c00f-43d9-885b-92fdce39128c,75d3fc52-3776-47d4-b7fd-8074d30b06d1,Where did I put the chopsticks.,"(0.0, 3.75)","(3.319, 4.619)",413fe086-1745-4573-b75b-e7d26ff72df9,beside the stove
1,1,00bb3571-8b35-40ba-8163-a1a0fd20b886,60e7e14d-cbed-46d1-924d-6ce451ea7d7c,What cable did I remove?,"(78.75, 86.25)","(84.32902, 84.71152)",03e90bbc-7d6b-423c-84d9-b5be3eff11c5,usb drive
2,2,d8fb0c3d-39b1-4bb2-b800-fc56f12ab120,e1c79556-e8af-4e26-bc4c-633100277239,Did I close the refrigerator?,"(0.0, 3.75)","(0.65536, 2.00097)",4ce119de-0f42-4bd1-b387-9e19643fdddc,yes
3,3,e8b1352a-6e0c-4f86-8a52-afe93743abb6,efc190a8-45de-4ce5-b480-b722403bcec1,Where was the scissors before I picked it up?,"(93.75, 97.5)","(87.0, 89.306)",ff6d3d52-dda5-46dd-8515-b9b772933030,on the plate rack
4,4,2bc5aa4c-3114-497e-9d13-e229277fe11b,cc6270fd-3c0d-4dda-bcb4-52cefc0224d7,In what location did I open the door?,"(3.75, 7.5)","(6.2829, 8.428)",432cb803-6be5-47bc-8443-6bb5b9051667,in the bathroom


Transform dataframe into structure for model.

### Install video-LLaVA

video-LLaVA model implemented as per [documentation](https://github.com/PKU-YuanGroup/Video-LLaVA).

To fit model on GPU and save memory usage it is quantizied as per [documentation](https://huggingface.co/docs/transformers/main/en/model_doc/video_llava).

Import all neccesary packages.

In [7]:
!git clone https://github.com/PKU-YuanGroup/Video-LLaVA
!cd Video-LLaVA
!conda create -n videollava python=3.10 -y
!conda activate videollava
!pip install --upgrade pip  # enable PEP 660 support
!pip install -e .
!pip install -e ".[train]"
!pip install decord opencv-python git+https://github.com/facebookresearch/pytorchvideo.git@28fe037d212663c6a24f373b94cc5d478c8c1a1d
!pip install --upgrade git+https://github.com/huggingface/transformers.git


fatal: destination path 'Video-LLaVA' already exists and is not an empty directory.
/bin/bash: line 1: conda: command not found
/bin/bash: line 1: conda: command not found
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-req-build-bn35aule
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-req-build-bn35aule
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f3

Define model.

In [8]:
!pip install bitsandbytes -U --prefer-binary

In [9]:
import av
import numpy as np
from transformers import VideoLlavaProcessor, VideoLlavaForConditionalGeneration, BitsAndBytesConfig
import torch


# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


# Initialize the model and processor once
model = VideoLlavaForConditionalGeneration.from_pretrained("LanguageBind/Video-LLaVA-7B-hf", quantization_config=quantization_config, device_map="auto")
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Define model usage functions.

In [10]:
import gc
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)

    #incase of too short video
    while len(frames) < 8:
        frames.append(frames[-1])

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

# Prepare data for Video-LLaVA
def prepare_qa_data(row, video_segments_dir):
    clip_uid = row['clip_uid']
    annotation_uid = row['annotation_uid']
    sentence = row['sentence']
    idx = int(row['idx'])
    video_segment_path = os.path.join(video_segments_dir, f"{idx}_{clip_uid}_{annotation_uid}.mp4")

    # Check if the video file exists
    if not os.path.exists(video_segment_path):
        print(f"Video file {video_segment_path} not found.")
        return None, None

    try:
        # Open the video file and check for video streams
        container = av.open(video_segment_path)
        if not container.streams.video:
            print(f"No video stream found in {video_segment_path}.")
            return None, None

        total_frames = container.streams.video[0].frames
        indices = np.arange(0, total_frames, total_frames / 8).astype(int)
        clip = read_video_pyav(container, indices)

        prompt = (
            "USER: You are given a visual summary of a video segment.\n"
            "<video>\n"
            "Answer the following question as precisely and concisely as possible.\n"
            "Answer concisely in 5 words or less.\n"
            f"Question: {sentence}\n"

        )
        inputs = processor(prompt, videos=clip)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        return inputs, prompt
    except Exception as e:
        print(f"Error processing video {video_segment_path}: {e}")
        return None, None

# Function to process a batch of data
def process_batch(batch_df):
    batch_answers = []
    for idx, row in batch_df.iterrows():

        inputs, prompt = prepare_qa_data(row, VIDEO_OUT_new_cut)
        if inputs is None:
            continue

        generate_ids = model.generate(**inputs, max_new_tokens=80)

        answer = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
        print(answer)

        times = row['predicted_times'].strip('()').split(",")
        batch_answers.append({
            'idx' : row['idx'],
            'annotation_uid' : row['annotation_uid'],
            'clip_uid' : row['clip_uid'],
            'sentence' : row['sentence'],
            'predicted_times' : row['predicted_times'],
            'exact_times' : row['exact_times'],
            'video_uid' : row['video_uid'],
            'ground truth' : row['ground truth'],
            'answer': answer.split("Answer:")[-1].strip() if "Answer:" in answer else answer.strip(),
        })

        # Clear variables to free up memory
        del inputs, generate_ids, answer
        gc.collect()
        torch.cuda.empty_cache()

    return batch_answers

Feed prompts and video into model.

In [11]:
import torch

In [13]:
# Process the data in batches
batch_size = 25
answers = []

for start in range(0, len(annotations_df), batch_size):
    end = min(start + batch_size, len(annotations_df))
    batch_df = annotations_df[start:end]
    answers.extend(process_batch(batch_df))

# Convert the answers to a DataFrame
answers_df = pd.DataFrame(answers)

# Save the answers to a CSV file
answers_csv_path = EXTENSION_DIR + '/video_qa_answers1.csv'
answers_df.to_csv(answers_csv_path, index=False)

# Display the first few rows of the answers DataFrame
answers_df.head()

USER: You are given a visual summary of a video segment.

Answer the following question as precisely and concisely as possible.
Answer concisely in 5 words or less.
Question: Where did I put the chopsticks.
Answer: In bowl
USER: You are given a visual summary of a video segment.

Answer the following question as precisely and concisely as possible.
Answer concisely in 5 words or less.
Question: What cable did I remove?
Answer: Power
USER: You are given a visual summary of a video segment.

Answer the following question as precisely and concisely as possible.
Answer concisely in 5 words or less.
Question: Did I close the refrigerator?
Answer: No
USER: You are given a visual summary of a video segment.

Answer the following question as precisely and concisely as possible.
Answer concisely in 5 words or less.
Question: Where was the scissors before I picked it up?
Answer: On counter
USER: You are given a visual summary of a video segment.

Answer the following question as precisely and co

,idx,annotation_uid,clip_uid,sentence,predicted_times,exact_times,video_uid,ground truth,answer
0,0,93891b44-c00f-43d9-885b-92fdce39128c,75d3fc52-3776-47d4-b7fd-8074d30b06d1,Where did I put the chopsticks.,"(0.0, 3.75)","(3.319, 4.619)",413fe086-1745-4573-b75b-e7d26ff72df9,beside the stove,In bowl
1,1,00bb3571-8b35-40ba-8163-a1a0fd20b886,60e7e14d-cbed-46d1-924d-6ce451ea7d7c,What cable did I remove?,"(78.75, 86.25)","(84.32902, 84.71152)",03e90bbc-7d6b-423c-84d9-b5be3eff11c5,usb drive,Power
2,2,d8fb0c3d-39b1-4bb2-b800-fc56f12ab120,e1c79556-e8af-4e26-bc4c-633100277239,Did I close the refrigerator?,"(0.0, 3.75)","(0.65536, 2.00097)",4ce119de-0f42-4bd1-b387-9e19643fdddc,yes,No
3,3,e8b1352a-6e0c-4f86-8a52-afe93743abb6,efc190a8-45de-4ce5-b480-b722403bcec1,Where was the scissors before I picked it up?,"(93.75, 97.5)","(87.0, 89.306)",ff6d3d52-dda5-46dd-8515-b9b772933030,on the plate rack,On counter
4,4,2bc5aa4c-3114-497e-9d13-e229277fe11b,cc6270fd-3c0d-4dda-bcb4-52cefc0224d7,In what location did I open the door?,"(3.75, 7.5)","(6.2829, 8.428)",432cb803-6be5-47bc-8443-6bb5b9051667,in the bathroom,Bathroom


In [12]:
!pip install rouge-score
!pip install evaluate
!pip install bert_score
!pip install nltk

In [15]:
import pandas as pd
import re
import evaluate
import torch
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Load answers DataFrame (make sure answers_df is already defined)
# Example:
# answers_df = pd.read_csv("your_answers.csv")

# Drop rows with missing values
answers_df = answers_df.dropna(subset=['ground truth', 'answer'])

# --- Tokenizer ---
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', str(text).lower())

# --- Tokenize ---
references = [[simple_tokenizer(ref)] for ref in answers_df['ground truth']]
hypotheses = [simple_tokenizer(pred) for pred in answers_df['answer']]

# --- BLEU Scores ---
smoothie = SmoothingFunction().method4
bleu1 = corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0), smoothing_function=smoothie)
bleu2 = corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
bleu3 = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
bleu4 = corpus_bleu(references, hypotheses, smoothing_function=smoothie)

print(f"\n🔵 BLEU Scores:")
print(f"BLEU-1: {bleu1:.4f}")
print(f"BLEU-2: {bleu2:.4f}")
print(f"BLEU-3: {bleu3:.4f}")
print(f"BLEU-4: {bleu4:.4f}")

# --- Prepare text for ROUGE & BERTScore ---
joined_hypotheses = [' '.join(tokens) for tokens in hypotheses]
joined_references = [' '.join(ref[0]) for ref in references]

# --- ROUGE ---
try:
    rouge = evaluate.load("rouge")
    rouge_results = rouge.compute(
        predictions=joined_hypotheses,
        references=joined_references
    )
    print("\n🔴 ROUGE Scores:")
    for k, v in rouge_results.items():
        print(f"{k.upper()}: {v:.4f}")
except Exception as e:
    print(f"\n❗ROUGE evaluation failed: {e}")
    rouge_results = {}

# --- BERTScore ---
try:
    bertscore = evaluate.load("bertscore")
    bertscore_results = bertscore.compute(
        predictions=joined_hypotheses,
        references=joined_references,
        lang="en"
    )
    avg_precision = sum(bertscore_results['precision']) / len(bertscore_results['precision'])
    avg_recall = sum(bertscore_results['recall']) / len(bertscore_results['recall'])
    avg_f1 = sum(bertscore_results['f1']) / len(bertscore_results['f1'])

    print("\n🟢 BERTScore (Semantic Similarity):")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall:    {avg_recall:.4f}")
    print(f"F1 Score:  {avg_f1:.4f}")
except Exception as e:
    print(f"\n❗BERTScore evaluation failed: {e}")
    avg_precision = avg_recall = avg_f1 = 0.0

# --- Save metrics summary ---
with open("/content/video_qa_metrics.txt", "w") as f:
    f.write(f"BLEU-1 Score: {bleu1:.4f}\n")
    f.write(f"BLEU-2 Score: {bleu2:.4f}\n")
    f.write(f"BLEU-3 Score: {bleu3:.4f}\n")
    f.write(f"BLEU-4 Score: {bleu4:.4f}\n")
    for k, v in rouge_results.items():
        f.write(f"ROUGE-{k.upper()}: {v:.4f}\n")
    f.write(f"BERTScore Precision: {avg_precision:.4f}\n")
    f.write(f"BERTScore Recall:    {avg_recall:.4f}\n")
    f.write(f"BERTScore F1:        {avg_f1:.4f}\n")


🔵 BLEU Scores:
BLEU-1: 0.1204
BLEU-2: 0.0221
BLEU-3: 0.0087
BLEU-4: 0.0043

🔴 ROUGE Scores:
ROUGE1: 0.1899
ROUGE2: 0.0075
ROUGEL: 0.1900
ROUGELSUM: 0.1922


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.9006
Recall:    0.8620
F1 Score:  0.8804


In [27]:
import re

def build_general_prompt(sentence: str) -> str:
    q = sentence.strip().lower()

    prompt = (
        "### SYSTEM:\n"
        "You are a helpful assistant that answers questions based on visual summaries of video segments.\n"
        "Use only the information visible in the video.\n"
        "Be accurate and respond in **5 words or fewer**.\n\n"

        "### CONTEXT:\n"
        "<video>\n"
        "(Visual summary of the video is shown here)\n\n"

        f"### USER:\nQuestion: {sentence.strip()}\n"
    )

    # -------------------------------
    # 🔍 Task Classification Logic
    # -------------------------------

    # 1. Counting tasks
    if re.search(r'\bhow many\b', q):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "This is a counting task. Count only the visible items.\n"
            "Return the number.\n"
        )

    # 2. Color detection
    elif re.search(r'\bwhat (color|colour)\b', q):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Identify the visible color of the mentioned object.\n"
        )

    # 3. Location questions (debiased)
    elif re.search(r'\bwhere (is|was|did|do)\b|\bin what location\b', q):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Identify the object's visible spatial location based only on the video summary.\n"
            "Use only location descriptions directly inferable from the visual context.\n"
        )

    # 4. Action-object identification
    elif re.search(r'\bwhat\b', q) and re.search(
        r'\b(remove|pick|grab|cut|wash|tie|wipe|press|use|carry|put|insert|drop|place)\b', q
    ):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Identify the object involved in the action based on what is clearly visible in the video.\n"
        )

    # 5. Tool/machine recognition
    elif re.search(r'\bwhat\b', q) and re.search(r'\b(tool|machine|device|object|item|equipment)\b', q):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Determine what object, tool, or machine was used. Base your answer only on visible cues.\n"
        )

    # 6. Human interaction
    elif re.search(r'\bwho did (i|he|she|we|they) (talk|interact)\b', q):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Name the person involved in the interaction.\n"
        )

    # 7. Yes/no — fallback only
    elif (
        re.search(r'\bdid\b.*\b(i|we|they|he|she)\b', q)
        and not re.search(r'\bwhat\b', q)
        and not re.search(r'\bwho\b', q)
    ):
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "This is a yes/no question. Respond with 'yes' or 'no' based on visible evidence only.\n"
        )

    # 8. Fallback
    else:
        prompt += (
            "\n### TASK INSTRUCTION:\n"
            "Answer briefly and accurately using only the information visible in the video summary.\n"
        )

    prompt += "\nAnswer:\n"
    return prompt

In [28]:
# Prepare data for Video-LLaVA
def prepare_qa_data(row, video_segments_dir):
    clip_uid = row['clip_uid']
    annotation_uid = row['annotation_uid']
    sentence = row['sentence']
    idx = int(row['idx'])
    video_segment_path = os.path.join(video_segments_dir, f"{idx}_{clip_uid}_{annotation_uid}.mp4")

    # Check if the video file exists
    if not os.path.exists(video_segment_path):
        print(f"Video file {video_segment_path} not found.")
        return None, None

    try:
        # Open the video file and check for video streams
        container = av.open(video_segment_path)
        if not container.streams.video:
            print(f"No video stream found in {video_segment_path}.")
            return None, None

        total_frames = container.streams.video[0].frames
        indices = np.arange(0, total_frames, total_frames / 8).astype(int)
        clip = read_video_pyav(container, indices)

        prompt = build_general_prompt(sentence)

        inputs = processor(prompt, videos=clip)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        return inputs, prompt
    except Exception as e:
        print(f"Error processing video {video_segment_path}: {e}")
        return None, None


In [29]:
# Process the data in batches
batch_size = 25
answers = []

for start in range(0, len(annotations_df), batch_size):
    end = min(start + batch_size, len(annotations_df))
    batch_df = annotations_df[start:end]
    answers.extend(process_batch(batch_df))

# Convert the answers to a DataFrame
answers_df = pd.DataFrame(answers)

# Save the answers to a CSV file
answers_csv_path = EXTENSION_DIR + '/video_qa_answers2.csv'
answers_df.to_csv(answers_csv_path, index=False)

# Display the first few rows of the answers DataFrame
answers_df.head()

### SYSTEM:
You are a helpful assistant that answers questions based on visual summaries of video segments.
Use only the information visible in the video.
Be accurate and respond in **5 words or fewer**.

### CONTEXT:

(Visual summary of the video is shown here)

### USER:
Question: Where did I put the chopsticks.

### TASK INSTRUCTION:
Identify the object's visible spatial location based only on the video summary.
Use only location descriptions directly inferable from the visual context.

Answer:
In the person's hand.
### SYSTEM:
You are a helpful assistant that answers questions based on visual summaries of video segments.
Use only the information visible in the video.
Be accurate and respond in **5 words or fewer**.

### CONTEXT:

(Visual summary of the video is shown here)

### USER:
Question: What cable did I remove?

### TASK INSTRUCTION:
Identify the object involved in the action based on what is clearly visible in the video.

Answer:
The person removed a black cable from the de

,idx,annotation_uid,clip_uid,sentence,predicted_times,exact_times,video_uid,ground truth,answer
0,0,93891b44-c00f-43d9-885b-92fdce39128c,75d3fc52-3776-47d4-b7fd-8074d30b06d1,Where did I put the chopsticks.,"(0.0, 3.75)","(3.319, 4.619)",413fe086-1745-4573-b75b-e7d26ff72df9,beside the stove,In the person's hand.
1,1,00bb3571-8b35-40ba-8163-a1a0fd20b886,60e7e14d-cbed-46d1-924d-6ce451ea7d7c,What cable did I remove?,"(78.75, 86.25)","(84.32902, 84.71152)",03e90bbc-7d6b-423c-84d9-b5be3eff11c5,usb drive,The person removed a black cable from the desk.
2,2,d8fb0c3d-39b1-4bb2-b800-fc56f12ab120,e1c79556-e8af-4e26-bc4c-633100277239,Did I close the refrigerator?,"(0.0, 3.75)","(0.65536, 2.00097)",4ce119de-0f42-4bd1-b387-9e19643fdddc,yes,No
3,3,e8b1352a-6e0c-4f86-8a52-afe93743abb6,efc190a8-45de-4ce5-b480-b722403bcec1,Where was the scissors before I picked it up?,"(93.75, 97.5)","(87.0, 89.306)",ff6d3d52-dda5-46dd-8515-b9b772933030,on the plate rack,On the table
4,4,2bc5aa4c-3114-497e-9d13-e229277fe11b,cc6270fd-3c0d-4dda-bcb4-52cefc0224d7,In what location did I open the door?,"(3.75, 7.5)","(6.2829, 8.428)",432cb803-6be5-47bc-8443-6bb5b9051667,in the bathroom,Bathroom


In [30]:
import pandas as pd
import re
import evaluate
import torch
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

# Load answers DataFrame (make sure answers_df is already defined)
# Example:
# answers_df = pd.read_csv("your_answers.csv")

# Drop rows with missing values
answers_df = answers_df.dropna(subset=['ground truth', 'answer'])

# --- Tokenizer ---
def simple_tokenizer(text):
    return re.findall(r'\b\w+\b', str(text).lower())

# --- Tokenize ---
references = [[simple_tokenizer(ref)] for ref in answers_df['ground truth']]
hypotheses = [simple_tokenizer(pred) for pred in answers_df['answer']]

# --- BLEU Scores ---
smoothie = SmoothingFunction().method4
bleu1 = corpus_bleu(references, hypotheses, weights=(1.0, 0, 0, 0), smoothing_function=smoothie)
bleu2 = corpus_bleu(references, hypotheses, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
bleu3 = corpus_bleu(references, hypotheses, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
bleu4 = corpus_bleu(references, hypotheses, smoothing_function=smoothie)

print(f"\n🔵 BLEU Scores:")
print(f"BLEU-1: {bleu1:.4f}")
print(f"BLEU-2: {bleu2:.4f}")
print(f"BLEU-3: {bleu3:.4f}")
print(f"BLEU-4: {bleu4:.4f}")

# --- Prepare text for ROUGE & BERTScore ---
joined_hypotheses = [' '.join(tokens) for tokens in hypotheses]
joined_references = [' '.join(ref[0]) for ref in references]

# --- ROUGE ---
try:
    rouge = evaluate.load("rouge")
    rouge_results = rouge.compute(
        predictions=joined_hypotheses,
        references=joined_references
    )
    print("\n🔴 ROUGE Scores:")
    for k, v in rouge_results.items():
        print(f"{k.upper()}: {v:.4f}")
except Exception as e:
    print(f"\n❗ROUGE evaluation failed: {e}")
    rouge_results = {}

# --- BERTScore ---
try:
    bertscore = evaluate.load("bertscore")
    bertscore_results = bertscore.compute(
        predictions=joined_hypotheses,
        references=joined_references,
        lang="en"
    )
    avg_precision = sum(bertscore_results['precision']) / len(bertscore_results['precision'])
    avg_recall = sum(bertscore_results['recall']) / len(bertscore_results['recall'])
    avg_f1 = sum(bertscore_results['f1']) / len(bertscore_results['f1'])

    print("\n🟢 BERTScore (Semantic Similarity):")
    print(f"Precision: {avg_precision:.4f}")
    print(f"Recall:    {avg_recall:.4f}")
    print(f"F1 Score:  {avg_f1:.4f}")
except Exception as e:
    print(f"\n❗BERTScore evaluation failed: {e}")
    avg_precision = avg_recall = avg_f1 = 0.0

# --- Save metrics summary ---
with open("/content/video_qa_metrics.txt", "w") as f:
    f.write(f"BLEU-1 Score: {bleu1:.4f}\n")
    f.write(f"BLEU-2 Score: {bleu2:.4f}\n")
    f.write(f"BLEU-3 Score: {bleu3:.4f}\n")
    f.write(f"BLEU-4 Score: {bleu4:.4f}\n")
    for k, v in rouge_results.items():
        f.write(f"ROUGE-{k.upper()}: {v:.4f}\n")
    f.write(f"BERTScore Precision: {avg_precision:.4f}\n")
    f.write(f"BERTScore Recall:    {avg_recall:.4f}\n")
    f.write(f"BERTScore F1:        {avg_f1:.4f}\n")


🔵 BLEU Scores:
BLEU-1: 0.2558
BLEU-2: 0.1607
BLEU-3: 0.0901
BLEU-4: 0.0333

🔴 ROUGE Scores:
ROUGE1: 0.2825
ROUGE2: 0.1282
ROUGEL: 0.2808
ROUGELSUM: 0.2801


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🟢 BERTScore (Semantic Similarity):
Precision: 0.8964
Recall:    0.8851
F1 Score:  0.8904
